# 🧩 Week 07-08 · Notebook 03 · Prompt Templates & Chain Composition

Engineer reusable prompt templates and compose LCEL chains that adapt to plant-specific policies.

## 🎯 Learning Objectives
- Build parameterized `ChatPromptTemplate` objects with manufacturing metadata.
- Compose LCEL chains that inject plant configuration (language, compliance grade).
- Create reusable evaluators that test prompt outputs before deployment.
- Capture template lineage for change-management approval.

## 🧩 Scenario
Corporate safety updated lockout/tagout wording. Prompts must absorb the change across plants without manual edits per notebook.

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Plant-specific configuration that can be dynamically loaded
plant_config = {
    "plant_id": "PUNE-01",
    "language": "en",
    "compliance_grade": "ISO 9001",
    "safety_disclaimer": "CRITICAL: Always follow Lockout/Tagout procedures (SOP-900) before maintenance. Verify zero energy state."
}

# Parameterized template that pulls from the config and runtime inputs
template_string = """
You are a Manufacturing Copilot for Plant {plant_id}.
Your response must adhere to {compliance_grade} standards.
Language: {language}

Safety Disclaimer: {safety_disclaimer}

---
Context from SOP:
{sop_snippet}

---
Question:
{question}

Provide a concise answer based ONLY on the provided SOP context.
"""

prompt = ChatPromptTemplate.from_template(template_string)

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
chain = prompt | llm | StrOutputParser()

# Invoke the chain with both config and runtime variables
answer = chain.invoke({
    "plant_id": plant_config["plant_id"],
    "language": plant_config["language"],
    "compliance_grade": plant_config["compliance_grade"],
    "safety_disclaimer": plant_config["safety_disclaimer"],
    "sop_snippet": "SOP-112, Section 4.1 states: 'The primary hydraulic pump requires filter replacement every 500 operating hours or 3 months, whichever comes first.'",
    "question": "How often do I need to replace the hydraulic pump filter?"
})

print(answer)

### 🧪 Template Regression Harness
Create an automated test to verify the disclaimer appears and citations reference SOP IDs.

In [ ]:
import re

def validate_response(text: str, config: dict) -> dict:
    """
    Validates the LLM response against key requirements.
    """
    disclaimer = config.get("safety_disclaimer", "")
    
    validation_results = {
        "has_disclaimer": disclaimer in text,
        "has_sop_citation": bool(re.search(r"SOP-\d+", text)),
        "is_concise": len(text.split()) < 75  # Example constraint
    }
    return validation_results

# Run validation
validation = validate_response(answer, plant_config)

# Assertions for automated testing
assert validation['has_disclaimer'], 'Safety disclaimer missing from response!'
assert validation['has_sop_citation'], 'SOP citation is missing from response!'

print("Validation Results:")
print(validation)

## 🔄 Template Versioning
| Field | Example | Purpose |
| --- | --- | --- |
| `template_id` | `PROMPT_MAINT_TRIAGE_V4` | Unique reference |

## 🧪 Lab Assignment
1. Localize the template to Spanish and verify disclaimers translate correctly.
2. Extend the chain with a retrieval step pulling the latest SOP snippet before answering.
3. Store template metadata in MLflow parameters when executing chains.
4. Capture test outputs and attach to the change-approval ticket.

## ✅ Checklist
- [ ] Prompt templates parameterized
- [ ] Regression harness created
- [ ] Template metadata logged
- [ ] Lab artefacts archived

## 📚 References
- LangChain Expression Language (LCEL) Guide
- Corporate Lockout/Tagout SOP-900
- MLflow Parameter Logging Examples